In [12]:
import os
from PIL import Image

# Set your target directory here
folder_path = "/Users/jonasmucke/Desktop/pose_estimation/Rat/bottom950/resized_images"
target_size = (1280, 720)
non_matching_count = 0

# Iterate through all files in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    
    try:
        with Image.open(file_path) as img:
            if img.size != target_size:
                print(f"{filename}: {img.size}")
                non_matching_count += 1
    except Exception as e:
        print(f"Skipping file {filename}: {e}")

print(f"\nTotal images not {target_size[0]}x{target_size[1]}: {non_matching_count}")



Total images not 1280x720: 0


In [16]:
import os
import cv2
import pandas as pd

# --- Config ---
image_folder = "/Users/jonasmucke/Desktop/pose_estimation/Rat/top950/images"
annotation_file = "/Users/jonasmucke/Desktop/pose_estimation/Rat/top950/annotations.csv"
output_annotation_file = "/Users/jonasmucke/Desktop/pose_estimation/Rat/top950/annotations_rescaled.csv"
output_image_folder = "/Users/jonasmucke/Desktop/pose_estimation/Rat/top950/resized_images"  # optionally save resized images elsewhere

# Create output folder if not exists
os.makedirs(output_image_folder, exist_ok=True)

# Load annotation CSV
df = pd.read_csv(annotation_file)

# Identify x and y columns
x_cols = [col for col in df.columns if col.endswith("-x")]
y_cols = [col for col in df.columns if col.endswith("-y")]

# Scaling factors
orig_size = (1920, 1200)
target_size = (1280, 720)
scale_x = target_size[0] / orig_size[0]  # 0.6667
scale_y = target_size[1] / orig_size[1]  # 0.6

# Keep track of which files were resized
resized_files = set()

# Go through the image folder
for filename in os.listdir(image_folder):
    if not filename.lower().endswith((".jpg", ".png", ".jpeg")):
        continue

    file_path = os.path.join(image_folder, filename)
    img = cv2.imread(file_path)
    
    if img is None:
        print(f"Skipping unreadable file: {filename}")
        continue

    h, w = img.shape[:2]

    if (w, h) == orig_size:
        # Resize image
        resized = cv2.resize(img, target_size)
        # Save resized image
        cv2.imwrite(os.path.join(output_image_folder, filename), resized)
        # Mark for annotation update
        resized_files.add(filename)
    else:
        # Copy image unchanged
        cv2.imwrite(os.path.join(output_image_folder, filename), img)

# Rescale annotations for resized files
df_scaled = df.copy()
df_scaled.loc[df_scaled['filename'].isin(resized_files), x_cols] *= scale_x
df_scaled.loc[df_scaled['filename'].isin(resized_files), y_cols] *= scale_y

# Save updated annotations
df_scaled.to_csv(output_annotation_file, index=False)
print(f"\nDone! Resized {len(resized_files)} images and updated annotations.")
print(f"Saved resized images to: {output_image_folder}")
print(f"Saved updated annotations to: {output_annotation_file}")



Done! Resized 151 images and updated annotations.
Saved resized images to: /Users/jonasmucke/Desktop/pose_estimation/Rat/top950/resized_images
Saved updated annotations to: /Users/jonasmucke/Desktop/pose_estimation/Rat/top950/annotations_rescaled.csv


In [18]:
import os
import random
import shutil

# --- Config ---
input_folder = "/Users/jonasmucke/Desktop/pose_estimation/Rat/top950/images"
output_base = "/Users/jonasmucke/Desktop/pose_estimation/Rat/top950/"  # where train/val/test folders will be created
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# Sanity check
assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6, "Ratios must sum to 1"

# Make sure output folders exist
train_dir = os.path.join(output_base, "train")
val_dir = os.path.join(output_base, "val")
test_dir = os.path.join(output_base, "test")

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Get all image files
image_files = [f for f in os.listdir(input_folder) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
random.shuffle(image_files)

# Split
n = len(image_files)
n_train = int(n * train_ratio)
n_val = int(n * val_ratio)

train_files = image_files[:n_train]
val_files = image_files[n_train:n_train + n_val]
test_files = image_files[n_train + n_val:]

# Helper to copy files
def copy_files(file_list, destination_folder):
    for f in file_list:
        src = os.path.join(input_folder, f)
        dst = os.path.join(destination_folder, f)
        shutil.copy2(src, dst)

# Copy
copy_files(train_files, train_dir)
copy_files(val_files, val_dir)
copy_files(test_files, test_dir)

print(f"Done! Split {n} images into:")
print(f"- {len(train_files)} train")
print(f"- {len(val_files)} val")
print(f"- {len(test_files)} test")
print(f"Saved in: {output_base}/")


Done! Split 950 images into:
- 760 train
- 95 val
- 95 test
Saved in: /Users/jonasmucke/Desktop/pose_estimation/Rat/top950//
